# Hybrid Model (CNN and Transformer)



In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, utils, Sequential
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.models import Model


### Reading The Data

In [2]:
training_data = utils.image_dataset_from_directory('the_wildfire_dataset/the_wildfire_dataset/train', image_size=(600,600))

validation_data = utils.image_dataset_from_directory('the_wildfire_dataset/the_wildfire_dataset/val', image_size=(600,600))

testing_data = utils.image_dataset_from_directory('the_wildfire_dataset/the_wildfire_dataset/test', image_size=(600,600))

Found 1887 files belonging to 2 classes.
Found 402 files belonging to 2 classes.
Found 410 files belonging to 2 classes.


### Perform Preprocessing

In [3]:
#Function to perform preprocessing where we normalize the images
def preprocessor(data, label):
    data = data/255.0
    return data,label

training_data = training_data.map(preprocessor)
validation_data = validation_data.map(preprocessor)
testing_data = testing_data.map(preprocessor)

In [4]:
#Class for building the simple hybrid model of CNN and Transformer
class HybridModel(Model):
    def __init__(self, num_classes=2):
        super(HybridModel, self).__init__()
        #CNN layer
        self.conv1 = layers.Conv2D(32, (3, 3), activation='relu', input_shape=(600, 600, 3))
        self.conv2 = layers.Conv2D(64, (3, 3), activation='relu')
        self.pool = layers.MaxPooling2D(pool_size=(2, 2))
        self.flatten = layers.Flatten()
        #transformer layer
        self.attention = MultiHeadAttention(num_heads=8, key_dim=256)
        self.dense1 = layers.Dense(256, activation='relu')
        self.dense2 = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.attention(x, x)
        x = self.dense1(x)
        outputs = self.dense2(x)
        return outputs

model = HybridModel()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
model.fit(training_data, validation_data=validation_data, epochs=2)

Epoch 1/10


In [8]:
def cnn_model_builder(input_shape):
    model = Sequential([])
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    
    # model.add(layers.Dropout(0.5))
    # model.add(layers.Dense(2, activation='softmax'))
    return model

In [9]:
input_shape = (600, 600, 3)
cnn_model = cnn_model_builder(input_shape)